In [1]:
import numpy as np
import pandas as pd

from clases.DenseLayer import DenseLayer
from clases.ConvLayer import ConvLayer
from clases.FlattenLayer import FlattenLayer
from clases.ActivationFunction import ReLU, Softmax
from clases.LossFunction import cross_entropy_loss, cross_entropy_loss_grad
from clases.Trainer import Trainer
from clases.Optimizer import GDOptimizer, AdamOptimizer
from clases.NeuronalNetwork import NeuronalNetwork

from clases.Test import *

In [2]:
# URL: https://www.kaggle.com/datasets/animatronbot/mnist-digit-recognizer
data = pd.read_csv('../datos/mnist_data.csv')
data = np.array(data)

In [3]:
m, n = data.shape
np.random.shuffle(data)

test_size = int(0.1 * m)
data_test = data[:test_size, :].T  # El 10% inicial para el conjunto de prueba

# El conjunto provisional restante (90%)
remaining_data = data[test_size:, :]

train_size = int(0.8 * m)
data_train = remaining_data[:train_size, :].T  # El siguiente 80% para entrenamiento
data_validation = remaining_data[train_size:, :].T  # El resto 10% para validación

X_train, Y_train = data_train[1:].T, data_train[0]
X_val, Y_val = data_validation[1:].T, data_validation[0]
X_test, Y_test = data_test[1:].T, data_test[0]

Y_train = np.eye(10)[Y_train]
Y_val = np.eye(10)[Y_val]
Y_test = np.eye(10)[Y_test]

# Normalización
X_train = X_train / 255
X_val = X_val / 255
X_test = X_test / 255

#Reshape
X_train = X_train.reshape(len(X_train), 1, 28, 28)

In [4]:
network = [
    ConvLayer((1,28,28),3,5),
    FlattenLayer((5,26,26), (1,5*26*26)),
    DenseLayer(5 * 26 * 26, 100, ReLU),
    DenseLayer(100, 10, Softmax)
]

In [ ]:
epochs = 20

for e in range(epochs):
    error = 0
    for x, y in zip(X_train, Y_train):
        output = x
        for layer in network:
            output = layer.forward(output)

        error += cross_entropy_loss(y, output)

        grad = cross_entropy_loss_grad(y, output)
        for layer in reversed(network):
            grad = layer.backward(grad)

    error /= len(X_train)
    print(f"{e+1}/{epochs}, error={error}")

1/20, error=0.344375153913375
2/20, error=0.2935703535872778
3/20, error=0.2774450412775601
4/20, error=0.26762219961211303
5/20, error=0.2614077211530698
6/20, error=0.25723308127150596
7/20, error=0.25410305554776785
8/20, error=0.25152565317872155
9/20, error=0.24926453238759125


In [ ]:
Y_pred = predict(nn, X_test)
accuracy, metrics = calculate_metrics(Y_pred, Y_test)
confusion_matrix(Y_test, Y_pred, class_labels=range(10))
print_metrics(accuracy,metrics)

In [ ]:
nn = NeuronalNetwork(None, [])
nn.layers = network
optimizer = GDOptimizer(learning_rate=0.1)
trainer = Trainer(nn, optimizer, cross_entropy_loss, cross_entropy_loss_grad)
loss, acc, val_info = trainer.train(X_train, Y_train,X_val,Y_val, epochs=20, print_every=1)

--------------------------------------------------------------------
0
1


IndexError: index 1 is out of bounds for axis 0 with size 1